In [ ]:
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
import os 
import shutil
import glob

In [ ]:
%cd /content/
%rm -rf darknet

/content


In [ ]:
!git clone https://github.com/AlexeyAB/darknet/
%cd darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 15298, done.
remote: Total 15298 (delta 0), reused 0 (delta 0), pack-reused 15298
Receiving objects: 100% (15298/15298), 13.68 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (10389/10389), done.
/content/darknet


In [ ]:
!pip install opencv-python==3.4.2.17

In [ ]:
!sed -i '1s/GPU=0/GPU=1/' Makefile 
!sed -i '2s/CUDNN=0/CUDNN=1/' Makefile 
!sed -i '4s/OPENCV=0/OPENCV=1/' Makefile 
!sed -i '6s/OPENMP=0/OPENMP=1/' Makefile 
!make
!chmod +x ./darknet

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -fopenmp -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./s

In [ ]:
# !cat Makefile

In [ ]:
%cp cfg/yolov3.cfg cfg/yolo-obj.cfg
!sed -i 's/batch=1/batch=64/g' cfg/yolo-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=32/g' cfg/yolo-obj.cfg
!sed -i 's/width=416/width=608/g' cfg/yolo-obj.cfg
!sed -i 's/height=416/height=608/g' cfg/yolo-obj.cfg

!sed -i '10s/channels=3/channels=1/' cfg/yolo-obj.cfg
!sed -i '20s/max_batches = 500200/max_batches = 2000/' cfg/yolo-obj.cfg
!sed -i '22s/steps=400000,450000/steps=1600,1800/' cfg/yolo-obj.cfg

!sed -i '610 s@classes=80@classes=1@' cfg/yolo-obj.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolo-obj.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolo-obj.cfg

!sed -i '603 s@filters=255@filters=18@' cfg/yolo-obj.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolo-obj.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolo-obj.cfg
# changes check
!cat cfg/yolo-obj.cfg

[net]
# Testing
batch=64
subdivisions=32
# Training
# batch=64
# subdivisions=326
width=608
height=608
channels=1
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 2000
policy=steps
steps=1600,1800
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutional]


In [ ]:
all_classes = """Cone
"""

file = """text_file = open("build/darknet/x64/data/obj.names", "w");text_file.write(all_classes);text_file.close()""" 

exec(file)

In [ ]:
dir = '/content/drive/My Drive/backup/'
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
%cp -a cfg/yolo-obj.cfg '/content/drive/My Drive/backup/'
%cp -a build/darknet/x64/data/obj.names '/content/drive/My Drive/backup/'

In [ ]:
!echo -e 'classes = 1\ntrain  = build/darknet/x64/data/train.txt\nvalid  = build/darknet/x64/data/test.txt\nnames = build/darknet/x64/data/obj.names\nbackup = /content/drive/My Drive/backup/' > build/darknet/x64/data/obj.data

In [ ]:
# check
!cat build/darknet/x64/data/obj.data
print()
!cat build/darknet/x64/data/obj.names

classes = 1
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/test.txt
names = build/darknet/x64/data/obj.names
backup = /content/drive/My Drive/backup/

Cone


In [ ]:
%mkdir build/darknet/x64/data/obj

In [ ]:
yolo_dir = '/content/drive/My Drive/yolov/dataset_grayscale'
if not os.path.exists(yolo_dir):
    os.makedirs(yolo_dir)


In [ ]:
!unzip '/content/drive/My Drive/yolov/dataset_grayscale.zip' -d '/content/drive/My Drive/yolov/'
%rm -rfv  '/content/drive/My Drive/yolov/dataset_grayscale.zip'

unzip:  cannot find or open /content/drive/My Drive/yolov/dataset_grayscale.zip, /content/drive/My Drive/yolov/dataset_grayscale.zip.zip or /content/drive/My Drive/yolov/dataset_grayscale.zip.ZIP.


In [ ]:
classes = """Cone"""

classes_names = """text_file = open("/content/drive/My Drive/yolov/dataset_grayscale/classes.txt", "w");
text_file.write(classes);
text_file.close()"""

exec(classes_names)

In [ ]:

# src_files = os.listdir(yolo_dir)
# for file_in_dir in src_files:
#     full_file_path = os.path.join(yolo_dir, file_in_dir) 
#     if os.path.isfile(full_file_path):
#        shutil.copy(full_file_path, target_dir)
#     print(file_in_dir)

In [ ]:
# if ls: cannot access 'build/darknet/x64/data/obj/*.txt': No such file or directory
target_dir = 'build/darknet/x64/data/obj'
for file_in_dir in glob.glob(os.path.join(yolo_dir,'*.*')):
    shutil.copy(file_in_dir, target_dir)

In [ ]:
# !find '/content/drive/My Drive/yolov/dataset_grayscale' -name '*.jpg' -exec cp -prv '{}' 'build/darknet/x64/data/obj' ';'
# !find '/content/drive/My Drive/yolov/dataset_grayscale' -name '*.txt' -exec cp -prv '{}' 'build/darknet/x64/data/obj' ';'

In [ ]:
%ls build/darknet/x64/data/obj/*.jpg | wc -l
%ls build/darknet/x64/data/obj/*.txt | wc -l

7619
7598


In [ ]:
import glob
import numpy as np
!echo -n > build/darknet/x64/data/train.txt
!echo -n > build/darknet/x64/data/test.txt
images = glob.glob('build/darknet/x64/data/obj/*.jpg')
print(images)

['build/darknet/x64/data/obj/vid_31_frame_1788.jpg', 'build/darknet/x64/data/obj/vid_42_frame_959.jpg', 'build/darknet/x64/data/obj/vid_203_frame_139.jpg', 'build/darknet/x64/data/obj/vid_64_frame_162.jpg', 'build/darknet/x64/data/obj/vid_79_frame_66.jpg', 'build/darknet/x64/data/obj/vid_31_frame_2170.jpg', 'build/darknet/x64/data/obj/vid_36_frame_117.jpg', 'build/darknet/x64/data/obj/vid_18_frame_1863.jpg', 'build/darknet/x64/data/obj/vid_18_frame_1752.jpg', 'build/darknet/x64/data/obj/vid_74_frame_95.jpg', 'build/darknet/x64/data/obj/vid_5_frame_1557.jpg', 'build/darknet/x64/data/obj/vid_2_frame_41877.jpg', 'build/darknet/x64/data/obj/vid_18_frame_1538.jpg', 'build/darknet/x64/data/obj/vid_31_frame_1843.jpg', 'build/darknet/x64/data/obj/vid_31_frame_2365.jpg', 'build/darknet/x64/data/obj/vid_3_frame_10730.jpg', 'build/darknet/x64/data/obj/vid_36_frame_252.jpg', 'build/darknet/x64/data/obj/vid_45_frame_456.jpg', 'build/darknet/x64/data/obj/vid_40_frame_926.jpg', 'build/darknet/x64/dat

In [ ]:
train_file = open("build/darknet/x64/data/train.txt", "w")
test_file = open("build/darknet/x64/data/test.txt", "w")

for j in images:
    if np.random.uniform(0,1) < 0.8:
        train_file.write(j + '\n')
    else:
        test_file.write(j + '\n')

train_file.close()
test_file.close()

In [ ]:
# check
!cat -n build/darknet/x64/data/train.txt | sed '1,5!d'
print()
!cat -n build/darknet/x64/data/test.txt | sed '1,5!d'

     1	build/darknet/x64/data/obj/vid_31_frame_1788.jpg
     2	build/darknet/x64/data/obj/vid_42_frame_959.jpg
     3	build/darknet/x64/data/obj/vid_64_frame_162.jpg
     4	build/darknet/x64/data/obj/vid_36_frame_117.jpg
     5	build/darknet/x64/data/obj/vid_18_frame_1863.jpg

     1	build/darknet/x64/data/obj/vid_203_frame_139.jpg
     2	build/darknet/x64/data/obj/vid_79_frame_66.jpg
     3	build/darknet/x64/data/obj/vid_31_frame_2170.jpg
     4	build/darknet/x64/data/obj/vid_18_frame_1538.jpg
     5	build/darknet/x64/data/obj/vid_31_frame_2365.jpg


In [ ]:
#Count number of files 
!wc -l build/darknet/x64/data/train.txt
!wc -l build/darknet/x64/data/test.txt

6113 build/darknet/x64/data/train.txt
1506 build/darknet/x64/data/test.txt


In [ ]:
!wget -P build/darknet/x64/ https://pjreddie.com/media/files/darknet53.conv.74
!find -name darknet53.conv.74 

--2021-08-05 14:30:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘build/darknet/x64/darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  19.9MB/s    in 8.6s    

2021-08-05 14:31:00 (18.1 MB/s) - ‘build/darknet/x64/darknet53.conv.74’ saved [162482580/162482580]

./build/darknet/x64/darknet53.conv.74


In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show -map

/bin/bash: ./darknet: No such file or directory


In [1]:
# Video test

In [ ]:
!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl
!youtube-dl -f '(bestvideo+bestaudio/best)'  -ciw -o "%(29-07-2021)s.%(mp4)s" -v 'https://www.youtube.com/watch?v=LVlxoYuyFQU&ab_channel=RaceyardKiel'

In [ ]:
config_file = '/content/drive/My Drive/backup/yolo-obj.cfg'
weight_file = '/content/drive/My Drive/backup/yolo-obj_best.weights'
class_names = '/content/drive/My Drive/backup/obj.names'
video_file = './NA.NA.mp4'

In [ ]:
import numpy as np
import cv2 
from google.colab.patches import cv2_imshow

In [ ]:
net = cv2.dnn.readNet(weight_file, config_file)

In [ ]:
classes = []
with open(class_names, 'r') as f:
    classes = f.read().splitlines()
classes

In [ ]:
cap = cv2.VideoCapture(video_file)
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255, size=(100,3))

In [ ]:
while True:
    _, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    blob = cv2.dnn.blobFromImage(gray, 1/255, (416, 416), 0, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayers()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.6:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.4)

    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(gray, (x,y), (x+w, y+h), color, 2)
            cv2.putText(gray, label + " " + confidence, (x, y+20), font, 2, (255,255,255), 2)

    cv2_imshow(gray)
    key = cv2.waitKey(1)
    if key==27:
        break

cap.release()
cv2.destroyAllWindows()